In [ ]:
%pip install U

Note: you may need to restart the kernel to use updated packages.


ERROR: You must give at least one requirement to install (see "pip help install")


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

In [25]:
# 1) Lectura inicial
df_varma = pd.read_csv(
    'Varma.csv',
    sep=',',
    header=None,
    skiprows=1,
    names=['ds', 'y', 'unique_id'],
    quoting=csv.QUOTE_NONE,
    engine='python'
)

In [26]:
# 2) Limpiar comillas sobrantes
df_varma['ds']       = df_varma['ds'].str.strip('"')
df_varma['unique_id']= df_varma['unique_id'].str.strip('"')

In [27]:
# 3) Convertir tipos
df_varma['ds'] = pd.to_datetime(df_varma['ds'], format='%Y%m')    # datetime64[ns]
df_varma['y']  = pd.to_numeric(df_varma['y'], errors='coerce').fillna(0)  # float

In [28]:
# 4) Completar series mensuales sin saltos
#   4.1 Orden y MultiIndex
df_varma = df_varma.sort_values(['unique_id','ds'])
df_varma = df_varma.set_index(['unique_id','ds'])

#   4.2 Generar índice completo (todos los IDs × cada mes del rango)
all_ids   = df_varma.index.get_level_values('unique_id').unique()
full_months = pd.date_range(
    start=df_varma.index.get_level_values('ds').min(),
    end  =df_varma.index.get_level_values('ds').max(),
    freq ='MS'
)
full_index = pd.MultiIndex.from_product(
    [all_ids, full_months],
    names=['unique_id','ds']
)

#   4.3 Reindexar rellenando y=0 donde faltara
df_complete = (
    df_varma
    .reindex(full_index, fill_value=0)
    .reset_index()   # volvemos a columnas “normales”
)

In [29]:
# 5) Asegurar orden final
df_complete = df_complete.sort_values(['unique_id','ds'], ignore_index=True)

In [30]:
# 6) Verificación
print(df_complete.dtypes)
# unique_id     object
# ds           datetime64[ns]
# y             float64
# dtype: object

print(df_complete.head(8))
#   unique_id         ds    y
# 0  0102300062 2023-01-01  3.0
# 1  0102300062 2023-02-01  1.0
# ...

print(df_complete.shape)  # p.ej. (n_series * n_meses, 3)

unique_id            object
ds           datetime64[ns]
y                   float64
dtype: object
    unique_id         ds     y
0  0102300062 2023-01-01   3.0
1  0102300062 2023-02-01   1.0
2  0102300062 2023-03-01   3.0
3  0102300062 2023-04-01   0.0
4  0102300062 2023-05-01   3.0
5  0102300062 2023-06-01  14.0
6  0102300062 2023-07-01  30.0
7  0102300062 2023-08-01   0.0
(10950, 3)


Qué consigues con esto:
1. Formato “largo” de tres columnas:
    - unique_id: identificador de la serie.
    - ds: fechas mensuales como datetime64[ns].
    - y: demanda numérica (float, con ceros para faltantes).

2. Serie completa y regular:
    - Cada (unique_id, ds) para todos los meses entre el mínimo y el máximo.
    - Relleno con y = 0 donde no había observación.

3. Orden y tipos correctos:
    - Orden ascendente por (unique_id, ds).
    - ds es datetime64[ns] y y es numérico.

---

In [33]:
# 1. Carga del Excel (ajusta sheet_name si no es la primera hoja)
file_path_noel = r"C:\OneDrive\OneDrive - IMPERIA SCM CONSULTING, S.L\MyPyProjects\Pro000\Noel_Test.xlsx"
df_excel = pd.read_excel(file_path_noel)

In [34]:
# 2. Renombrar columnas si es necesario
#    Supongamos que tus columnas originales se llaman: 'fecha', 'demanda', 'producto'
#    Ajusta estos nombres al leer tu Excel real.
df_excel = df_excel.rename(columns={
    'CODE_MONTH': 'ds',
    'TOTAL_AMOUNT': 'y',
    'PRODUCT_CODE': 'unique_id'
})

In [35]:
# 3. Limpiar comillas (si las hubiera)
df_excel['ds']        = df_excel['ds'].astype(str).str.strip('"')
df_excel['unique_id'] = df_excel['unique_id'].astype(str).str.strip('"')

In [36]:
# 4. Convertir tipos
df_excel['ds'] = pd.to_datetime(df_excel['ds'], format='%Y%m')  # o ajusta el formato según tu Excel
df_excel['y']  = pd.to_numeric(df_excel['y'], errors='coerce').fillna(0)

In [37]:
# 5. Completar series mensuales
df_excel = df_excel.sort_values(['unique_id','ds'])
df_excel = df_excel.set_index(['unique_id','ds'])

all_ids    = df_excel.index.get_level_values('unique_id').unique()
full_months = pd.date_range(
    start=df_excel.index.get_level_values('ds').min(),
    end  =df_excel.index.get_level_values('ds').max(),
    freq ='MS'
)
full_index = pd.MultiIndex.from_product(
    [all_ids, full_months],
    names=['unique_id','ds']
)

df_excel_complete = (
    df_excel
    .reindex(full_index, fill_value=0)
    .reset_index()
    .sort_values(['unique_id','ds'], ignore_index=True)
)

In [38]:
# 6. Verificación
print(df_excel_complete.dtypes)
print(df_excel_complete.head())
print("Shape:", df_excel_complete.shape)

unique_id            object
ds           datetime64[ns]
y                   float64
dtype: object
  unique_id         ds         y
0     12193 2019-01-01  17148.81
1     12193 2019-02-01  18449.09
2     12193 2019-03-01  16504.88
3     12193 2019-04-01  16862.63
4     12193 2019-05-01      0.00
Shape: (24776, 3)


---

In [40]:
# GUARDADO DE LOS DF OBTENIDOS
df_complete.to_excel('df_varma.xlsx', index=False)
df_excel_complete.to_excel('df_noel.xlsx', index=False)

---

# Intermittent or Sparse Data

## Exploración

Cada serie temporal representa las demandas de un producto específico. A este nivel, mayoría de los datos es intermitnete.

In [ ]:
#